## Using OpenKIM models in LAMMPS

This notebook demonstrates use of OpenKIM models in LAMMPS and some utilities available as LAMMPS subcommands. Documentation on the implementation of these features in LAMMPS can be found here: [LAMMPS `kim` command](https://docs.lammps.org/kim_commands.html). Subsections of this documentation are referenced thoughout the notebook where relevant.

For comparison, we begin with a non-KIM LAMMPS script for a single energy/force evaluation (i.e. a `run 0` script) of bulk silicon using the 1985 Stillinger-Weber potential. A user running this simulation would need to obtain their own parameter file (this one is shipped with LAMMPS) and specify the lattice constant in the input script:

<button data-commandLinker-command="docmanager:open" data-commandlinker-args='{"path":"openkim-demo/lammps/in.Si.sw.lammps"}'>View `in.Si.sw.lammps` in editor</button>

Run the computation:

In [ ]:
!lmp -in in.Si.sw.lammps

To contextualize these simple calculations, we can visualize the structure we are evaluating the forces and energies on. The box indicates the simulation domain, atoms outside the box are repeated for clarity.

In [ ]:
from ase.io import read
from ase.visualize import view
atoms = read('Si.data.tmp',format="lammps-data",style="atomic",Z_of_type={1:14}).repeat(2)
atoms.set_cell(0.5*atoms.get_cell())
view(atoms,viewer='ngl')

Now let's look at the same simulation set up with OpenKIM. Initialization and setup of the atomic interactions is handled by the `kim init` and `kim interactions` commands ([LAMMPS documentation](https://docs.lammps.org/kim_commands.html#using-openkim-ims-with-lammps-kim-init-kim-interactions))

<button data-commandLinker-command="docmanager:open" data-commandlinker-args='{"path":"openkim-demo/lammps/in.Si.sw.kim.lammps"}'>View `in.Si.sw.kim.lammps` in editor</button>

Run the computation:

In [ ]:
!lmp -in in.Si.sw.kim.lammps

Differences in the KIM and LAMMPS implementation of the Stillinger-Weber potential cause small differences numerical differences, but the results of the two computations are nearly identical. Note also that KIM prints brief citation information for the potential to the screen and detailed BibTeX-formatted citations to the LAMMPS log file.

You may have noticed that there is a small, but finite pressure in this configuration. This is because the 5.43 angstrom value we used for the lattice constant is not exactly what the Stillinger-Weber potential predicts. We can use the `kim query` LAMMPS command ([LAMMPS documentation](https://docs.lammps.org/kim_commands.html#using-openkim-web-queries-in-lammps-kim-query)) to obtain the correct equilibrium lattice constant for this potential from the OpenKIM repository:

<button data-commandLinker-command="docmanager:open" data-commandlinker-args='{"path":"openkim-demo/lammps/in.Si.sw.kim.query.lammps"}'>View `in.Si.sw.kim.query.lammps` in editor</button>

More information about querying OpenKIM can be found [here](https://openkim.org/doc/usage/kim-query/).

If we run the simulation, we see that the pressure is now nearly zero and the energy is slightly lower:

In [ ]:
!lmp -in in.Si.sw.kim.query.lammps

Let's try to run the same bulk silicon calculation with a different OpenKIM model, in this case a ReaxFF potential:

<button data-commandLinker-command="docmanager:open" data-commandlinker-args='{"path":"openkim-demo/lammps/in.Si.reax.kim.query.lammps"}'>View `in.Si.sw.kim.query.lammps` in editor</button>

In [ ]:
!lmp -in in.Si.reax.kim.query.lammps

As you can see, there is an error because the potential is not compatible with `metal` units. 

> **_NOTE:_** Natively, LAMMPS has support for unit hints in parameter file comments, in which case LAMMPS will also raise an error, or perform a unit conversion for certain pair styles. If the potential file does not include these optional comments, LAMMPS will have no native way of knowing the units are incorrect, and will give incorrect results.

Most KIM portable models are capable of internal unit conversion and will work with whatever units the user requests (see [Types of KIM Content](https://openkim.org/doc/repository/kim-content/) for an explanation of different types of models). If a model is not capable of internal unit conversion, a LAMMPS script using a KIM model can still be made to work with any LAMMPS units using the `unit_conversion_mode` option for `kim init` [LAMMPS documentation](https://docs.lammps.org/kim_commands.html#using-openkim-ims-with-lammps-kim-init-kim-interactions). This feature initializes a number of *internal* style LAMMPS variables that convert *from* the units the user requested *to* the units the model uses. With these variables, scripts can specify inputs and produce outputs in the desired system of units regardless of the model being used. This is important for writing fully model-agnostic LAMMPS inputs, such as those used for KIM Tests.

Here is the previous script using ReaxFF modified using `unit_conversion_mode` to use `metal` units:

<button data-commandLinker-command="docmanager:open" data-commandlinker-args='{"path":"openkim-demo/lammps/in.Si.reax.kim.query.unit.lammps"}'>View `in.Si.sw.kim.query.unit.lammps` in editor</button>

When we run the script, we see that the converted energy value is similar to the Stillinger-Weber result, indicating that we are using correct units:

In [ ]:
!lmp -in in.Si.reax.kim.query.unit.lammps

### Exercise

Compute the pressure of bulk silicon under 1% compression using the two KIM models we've used, correctly converting units. Which model has the higher bulk modulus?

> **_HINTS:_** The *thermo keyword* for pressure (equivalent to `pe` for potential energy) is `press`. The KIM unit conversion factor for pressure is `_u_pressure`. To create the compression, multiply the equilibrium lattice constant by 0.99. You can create nearly identical scripts for both models and only change the model name, since `unit_conversion_mode` can be used with any model. Answers are in the `lammps/answers` directory. You can create a new file by right-clicking in the browser on the left-hand side, in the launcher by opening a tab, or through the *File* menu. You can run LAMMPS in a new cell in this notebook, or by opening a terminal through the launcher or *File* menu.